<a href="https://colab.research.google.com/github/sanchit2843/MusicGenreClassification/blob/master/exampleprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1yV10gYOnepfmSj_08g0gHzstovg7X52q',
                                    dest_path='/content/dense161.h5')

In [0]:
!git clone https://github.com/sanchit2843/MusicGenreClassification
%cd MusicGenreClassification

Cloning into 'MusicGenreClassification'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 216 (delta 49), reused 46 (delta 10), pack-reused 99
Receiving objects: 100% (216/216), 1.02 MiB | 20.00 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/MusicGenreClassification


In [0]:
!pip install efficientnet_pytorch # Required to install
from model import dense161
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = dense161().to(device)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth
100%|██████████| 115730790/115730790 [00:01<00:00, 75583904.70it/s]


In [0]:
from torch import nn
model.load_state_dict(torch.load('/content/dense161.h5', map_location=lambda storage, loc: storage))
model.eval() #To set model in evaluation model
sm = nn.Softmax()

In [0]:
#Upload audio file and pass path to audio file
import os
path = '/content/Lately_Kind_of_Yeah_-_08_-_Aeronaut.mp3'
#Output path for spectograms
os.makedirs('/content/spec')
out_path = '/content/spec'

In [0]:
from Data.data_preprocessing import Dataset

In [36]:
#Pass output path where u want to save the spectograms
audio_data = Dataset()
audio_data.spec_create(path,out_path)

/usr/local/lib/python3.6/dist-packages/matplotlib/tight_layout.py:231: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


In [0]:
#Label decoder used at the time of training
decoder = {}
decoder[0] ='Hip-Hop' 
decoder[1] = 'Pop'
decoder[2] = 'Rock'
decoder[3] = 'Experimental'
decoder[4] = 'Folk'
decoder[5] = 'Instrumental'
decoder[6] = 'Electronic'
decoder[7] = 'International'

In [0]:
#image transformations
from torchvision import transforms
im_size = 128
train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])

In [0]:
#Predicting for 10 spectograms created by 1 audio file
import cv2
import os
from torch import nn

def pred(img_path):
  pred = list()
  for i in range(10):
    p1 = os.path.join(img_path,'spec_{}.png'.format(i))
    img = cv2.imread(p1)
    img = train_transforms(img)
    img = img.expand(1,-1,-1,-1)
    prediction = sm(model(img))
    pred.append(int(prediction.detach().argmax(1)))
  return pred

In [0]:
def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
    return num 

In [41]:
predictions = pred(out_path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


In [42]:
print('Predicted Genre is:', decoder[most_frequent(predictions)])

Predicted Genre is: Experimental
